In [101]:
# common workflow for me is:
# copy video from osmo action to desktop (assuming vaapi),
# and try compress it without lossing too much quality,
# and copy to NAS.
# TODO:
# 1. transcode in desktop or NAS?
# 2. GPU or CPU to transcode?

import os
import pathlib
import json

import ffmpeg
import subprocess

In [102]:
def get_video_info(file_path):
    probe = ffmpeg.probe(file_path)
    video_info = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    return video_info

def video_info_highlight(video_info):
    ret = {}
    ret['codec'] = video_info['codec_name']
    ret['resolution'] = str(video_info['width']) + 'x' + str(video_info['height'])
    ret['pixel'] = video_info['pix_fmt']
    ret['frame_rate'] = eval(video_info['avg_frame_rate'])
    ret['color_space'] = video_info['color_space']
    return ret

def get_audio_stream(file_path):
    probe = ffmpeg.probe(file_path)

    # Find audio stream and extract bitrate
    audio_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)
    return audio_stream
    
def audio_info_highlight(audio_stream):
    pass

# vmaf, ssim, psnr
def measure():
    pass

In [103]:
import ffmpeg

def transcode_new(input_video, output_video, video_opts, first_minute_only):
    
    # Define audio codec options
    audio_opts = {
        'c:a': 'aac',
        'b:a': '128k',
    }

    if first_minute_only:
        input_stream = ffmpeg.input(input_video, hwaccel='dxva2', ss='00:00:00', t='00:01:00')
    else:
        input_stream = ffmpeg.input(input_video, hwaccel='dxva2')

    output_stream = (
        ffmpeg
        .output(input_stream.video, input_stream.audio, output_video, **video_opts, **audio_opts)
    )

    cmd = ffmpeg.compile(output_stream)
    return cmd


In [106]:
nvenc_opts = {
    'vf': 'scale=1920:-1',
    # 'pix_fmt': f'yuv422p{bit_depth}le' if is_hdr else f'yuv420p{bit_depth}le',
    'c:v': 'hevc_nvenc',
    'rc': 'vbr',
    'qmin': 10,
    'qmax': 28,
    'preset': 'medium',
    'level': 5.1,
}

x265_opts = {
    'vf': 'scale=1920:-1',
    # 'pix_fmt': f'yuv422p{bit_depth}le' if is_hdr else f'yuv420p{bit_depth}le',
    'c:v': 'libx265',
    'crf': 22,
    'preset': 'medium',
}

osmo_opts = {
    'vf': 'scale=1920:-1',
    # 'pix_fmt': f'yuv422p{bit_depth}le' if is_hdr else f'yuv420p{bit_depth}le',
    'c:v': 'hevc_nvenc',
    'rc': 'vbr',
    'qmin': 10,
    'qmax': 28,
    'preset': 'medium',
    'level': 5.1,
}

dir = r'E:\OSMO'
files = os.listdir(dir)

cmds = []

# Print all files
for file in files:
    cmds.append(' '.join(transcode_new(dir+'\\'+file, dir+'\\'+file+'.transcoded.mkv', osmo_opts, False)))

In [107]:
import sys

sys.path.append('../../python3')

from command_worker_pool import WorkerPool

w = WorkerPool(5, cmds)
w.join()



Executing command: ffmpeg -hwaccel dxva2 -i DJI_0012.MP4 -map 0:v -map 0:a -b:a 128k -c:a aac -c:v hevc_nvenc -level 5.1 -preset medium -qmax 28 -qmin 10 -rc vbr -vf scale=1920:-1 DJI_0012.MP4.transcoded.mkv
Executing command: ffmpeg -hwaccel dxva2 -i DJI_0012_001.MP4 -map 0:v -map 0:a -b:a 128k -c:a aac -c:v hevc_nvenc -level 5.1 -preset medium -qmax 28 -qmin 10 -rc vbr -vf scale=1920:-1 DJI_0012_001.MP4.transcoded.mkv
Executing command: ffmpeg -hwaccel dxva2 -i DJI_0012_002.MP4 -map 0:v -map 0:a -b:a 128k -c:a aac -c:v hevc_nvenc -level 5.1 -preset medium -qmax 28 -qmin 10 -rc vbr -vf scale=1920:-1 DJI_0012_002.MP4.transcoded.mkv
Executing command: ffmpeg -hwaccel dxva2 -i DJI_0012_003.MP4 -map 0:v -map 0:a -b:a 128k -c:a aac -c:v hevc_nvenc -level 5.1 -preset medium -qmax 28 -qmin 10 -rc vbr -vf scale=1920:-1 DJI_0012_003.MP4.transcoded.mkv
Executing command: ffmpeg -hwaccel dxva2 -i DJI_0013.MP4 -map 0:v -map 0:a -b:a 128k -c:a aac -c:v hevc_nvenc -level 5.1 -preset medium -qmax 28